In [ ]:
import arablepy
client = arablepy.ArableClient()
from   sensor_includes   import email, password_ccber
client.connect(email = email, password = password_ccber)

from   datetime          import timedelta
from   datetime          import datetime
from   io                import StringIO
import os

import pandas as pd

pd.options.display.max_columns = None
pd.set_option('max_colwidth', 40)

data_dictionary = dict()
available_tables = client.schema(df = False)
available_tables

# description, data_type, column_name
for table in available_tables:
    try:
        table_data_dictionary = client.schema(table, df=True)
        table_data_dictionary_columns = table_data_dictionary["column_name"]
        table_data_dictionary = table_data_dictionary.drop("column_name", axis="columns")
        table_data_dictionary = table_data_dictionary.drop("data_type", axis="columns")
        table_data_dictionary = table_data_dictionary.set_index(table_data_dictionary_columns)
        table_data_dictionary = table_data_dictionary["description"].to_dict()
        data_dictionary[table] = table_data_dictionary
    except: # HTTPError - 500 Server Error: Internal Server Error for url
        continue


devices = client.devices(df=True)
device_locations = devices["current_location"]
test = dict()
for key in list(device_locations[0].keys()):
    try:
        device_location_names = [i[key] for i in device_locations]
        test[key] = device_location_names
    except KeyError:
        continue
device_data = pd.DataFrame.from_dict(test)

device_data = device_data[device_data["addr_state"] == "CA"]
device_data = device_data[device_data["name"] != "Carpinteria"]


def find_dates(index: int, option: str):
    start = datetime.strptime(index, "%Y-%m-%dT%H:%M:%S")
    end = datetime.now()
    dates = []

    for i in range(1, 12):
        for j in [start.year, end.year]:
            date = start.replace(year = j).replace(month = i + 1).replace(day = 1
                   ).replace(hour = 0).replace(minute = 0).replace(second = 0)
            if start <= date <= datetime.now():
                dates.append(date)
                
    if option == "start":
        dates.insert(0, start)
    elif option == "end":
        dates.insert(-1, end)
        
    dates = sorted([date.strftime("%Y-%m-%dT%H:%M:%S") for date in dates])
    dates = [f"{date}Z" for date in dates]
    return dates

def find_dates_years(index: int, option: str):
    start = datetime.strptime(index, "%Y-%m-%dT%H:%M:%S")
    end = datetime.now()
    dates = []

    for j in [start.year, end.year]:
        date = start.replace(year = j).replace(month = 1).replace(day = 1
                   ).replace(hour = 0).replace(minute = 0).replace(second = 0)
        if start <= date <= datetime.now():
            dates.append(date)
                
    if option == "start":
        dates.insert(0, start)
    elif option == "end":
        dates.insert(-1, end)
        
    dates = sorted([date.strftime("%Y-%m-%dT%H:%M:%S") for date in dates])
    dates = [f"{date}Z" for date in dates]
    return dates

def VWC(df):
    raw_VWC = df['sdi12_value_0'].to_list()
    calc_VWC = []
    for raw in raw_VWC:
        raw = float(raw)
        calc = 4.3000 * 10**(-6) * raw**(3) 
        - 5.50 * 10**(-4) * raw**(2) 
        + 2.92 * 10**(-2) * raw - 5.30 * 10**(-2)
        calc_VWC.append(calc)
    return calc_VWC

def time_and_date(df):
    tm = df['time'].to_list()
    date_stamp = []
    time_stamp = []
    for i in range(len(tm)):
        date_stamp.append(str(tm[i])[:10])
        time_stamp.append(str(tm[i]))
    return date_stamp, time_stamp

start_dates = []
end_dates = []
for i in list(device_data["start_date"]):
    try:
        start_dates.append(find_dates_years(i[:19], "start"))
        end_dates.append(find_dates_years(i[:19], "end"))
    except:
        continue

def retrieve_dataframe(data_type, data_type_name, device_site, device_site_acronym, device_name, device_id, start_time, end_time):
    """
    cursor (string)          : Encoded cursor token (for pagination, from X-Cursor-Next response header)
    limit (integer)          : [ 1...10000 ]; Default: 1000
    order (string)           : Default: "asc". Enum: "asc" "desc"
    temp (string)            : Enum: "C" "F". Temperature unit in either [C]elsius or [F]ahrenheit
    pres (string)            : Enum: "mb" "kp" Pressure unit in either millibars [mb] or kilopascals [kp]
    ratio (string)           : Enum: "dec" "pct" Ratio either as percent [pct] or decimal value [dec]
    size (string)            : Enum: "in" "mm" Size unit in either [in]ches or millimeters [mm]
    speed (string)           : Enum: "mps" "kph" "mph" Speed unit; meters per second [mps], 
                               kilometers per hour [kph], or miles per hour [mph]
    device (string)          : Device name, e.g., A000123 (required if location not specified)
    location (string)        : Location ID (required if device not specified)
    local_time (string)      : Local time column specified as timezone name, offset seconds or 
                               ISO format(e.g. America/Los_Angeles, -14400, -10: 30)(optional)
    select (Array of strings): Comma-separated column list, e.g., time,device,location,tair
    start_time (string)      : Start date/time, e.g., 2019-01-01 or 2019-01-01T00:00:00Z
    end_time (string)        : End date/time, e.g., 2019-01-01 or 2019-01-01T00:00:00Z
    """
    main_path = "/mnt/c/Users/15303/Documents"# "C:/Users/15303/Documents" # "G:\My Drive\CCBER"
    parent_folder = '/'.join([main_path, "Arable_Data", '-'.join([i.capitalize() for i in device_site.split()]), '-'.join([i.capitalize() for i in device_name.split()]), start_time[:4], data_type_name])
    if not os.path.exists(parent_folder):
        os.makedirs(parent_folder)
        print(f"Directory {parent_folder} created")
    else:    
        print(f"Directory {parent_folder} already exists")  
        pass
        
    df = client.data(data_type, 
                     devices = [device_id], 
                     start_time = start_time,
                     end_time = end_time)
    
    df = df.dropna(axis = 'columns', thresh=2)  # how='all')
    
    try:
        time_stamp, date_stamp = time_and_date(df)
        df.insert(1, 'Time', time_stamp)
        df.insert(1, 'Date', date_stamp) 
        df.insert(1, 'Habitat', '-'.join(device_name.split()))
        df.insert(1, 'Site', '-'.join(device_site.split()))
    except:
        pass
    try:
        df = df.rename(mapper = data_dictionary[data_type], axis='columns') 
    except:
        pass
        
    columns_to_remove    = {"daily": ['location','time','lat','long'],
                            "hourly": ['location','time','lat','long'],
                            "aux_raw": ['location','sdi12_version','sdi12_vendor_id','sdi12_sensor_model','sdi12_sensor_version','sdi12_sensor_sn'],
                            "health": ['location','time','lat','long'],
                            "local_hourly": ['location','time','lat','long'], 
                            "location_irrigation_forecast_daily": ['location','time','lat','long'], 
                            "sentek_daily": ['location','time','lat','long'], 
                            "sentek_hourly": ['location','time','lat','long']}
    
    for i in columns_to_remove[data_type]:
        try:                
            df = df.drop(i, axis='columns')
        except:
            pass
            
    if data_type == "aux_raw":
        try:
            calc_VWC = VWC(df)
            df.insert(6, '5TE Calibrated VWC (m^3/m^3)', calc_VWC)
        except:
            print("Failed to replace raw data with calculated data!")
    
            
    file_name = f"{parent_folder}/{'-'.join([i.capitalize() for i in device_name.split()])}_{data_type_name}_{start_time[:7].split('-')[0]}.csv"
    df.to_csv(file_name)
    print(f"Wrote {file_name}")
    
device_names = list(device_data["name"])
device_ids = list(device_data["device_name"])
device_sites = [sorted(i, key=len)[-1] for i in device_data["tags"]]
device_sites_acronyms = [sorted(i, key=len)[0] for i in device_data["tags"]]
data_names = ['soil', 'daily', 'health', 'hourly', 'local_hourly', 
              'location_irrigation_forecast_daily', 
              'sentek_daily', 'sentek_hourly']
data_types = available_tables

for t in range(len(data_types)):
    data_name = data_names[t]
    data_type = data_types[t]
    for n in range(len(device_names)):
        name = device_names[n]
        site = device_sites[n]
        site_acronym = device_sites_acronyms[n]
        ids = device_ids[n]
        for d in range(len(start_dates[n])):
            retrieve_dataframe(data_type, data_name, site, site_acronym, name, ids, start_dates[n][d], end_dates[n][d])
